In [3]:
import tensorflow as tf
import pandas as pd
import tensorflow.feature_column as fc
from tensorflow.keras.datasets import boston_housing

In [8]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [9]:
features = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATION', 'B', 'LSTAT']

In [10]:
x_train_df = pd.DataFrame(x_train, columns = features)
x_test_df = pd.DataFrame(x_test, columns = features)
y_train_df = pd.DataFrame(y_train, columns = ['MEDV'])
y_test_df = pd.DataFrame(y_test, columns = ['MEDV'])
x_train_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATION,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65


In [13]:
feature_columns = []
for feature_name in features:
    feature_columns.append(fc.numeric_column(feature_name, dtype = tf.float32))

In [24]:
def estimator_input(df_data, df_label, epochs=10, shuffle= True, batch_size = 32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(df_data), df_label))
        if shuffle:
            ds =ds.shuffle(100)
        ds = ds.batch(batch_size).repeat(epochs)
        return ds
    return input_function
train_fn = estimator_input(x_train_df, y_train_df)
val_input = estimator_input(x_test_df, y_test_df, epochs = 1, shuffle = False)
    

In [25]:
linear_reg = tf.estimator.LinearRegressor(feature_columns= feature_columns)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\olasu\\AppData\\Local\\Temp\\tmpt_1uk9_8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [26]:
linear_reg.train(train_fn, steps = 100)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.


C:\Users\olasu\anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\olasu\AppData\Local\Temp\tmpt_1uk9_8\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 681.08997, step = 0
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 100...
INFO:tensorflow:Saving checkpoints for 100 into C:\Users\olasu\AppData\Local\Temp\tmpt_1uk9_8\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 100...
INFO:tensorflow:Loss for final step: 78.38058.


In [27]:
result = linear_reg.evaluate(val_input)

INFO:tensorflow:Calling model_fn.


C:\Users\olasu\anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-07-07T14:10:13
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\olasu\AppData\Local\Temp\tmpt_1uk9_8\model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.76938s
INFO:tensorflow:Finished evaluation at 2022-07-07-14:10:13
INFO:tensorflow:Saving dict for global step 100: average_loss = 69.649025, global_step = 100, label/mean = 23.078432, loss = 82.84137, prediction/mean = 21.941593
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: C:\Users\olasu\AppData\Local\Temp\tmpt_1uk9_8\model.ckpt-100


In [30]:
result = linear_reg.predict(val_input)
for pred, exp in zip(result, y_test [:32]):
    print ("predicted value ", pred['predictions'] [0], "Expected ", exp)

INFO:tensorflow:Calling model_fn.


C:\Users\olasu\anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\canned\linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\olasu\AppData\Local\Temp\tmpt_1uk9_8\model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
predicted value  3.0731335 Expected  7.2
predicted value  22.492245 Expected  18.8
predicted value  21.04154 Expected  19.0
predicted value  22.677681 Expected  27.0
predicted value  21.631985 Expected  22.2
predicted value  20.812168 Expected  24.5
predicted value  28.886518 Expected  31.2
predicted value  24.76251 Expected  22.9
predicted value  19.86015 Expected  20.5
predicted value  22.625574 Expected  23.2
predicted value  8.856479 Expected  18.6
predicted value  22.00759 Expected  14.5
predicted value  22.226076 Expected  17.8
predicted value  21.982841 Expected  50.0
predicted value  17.279495 Expected  20.8
predicted value  23.952738 Expected  24.3
predicted value  19.667007 Expected  24.2
predicted value  20.41338 Expe